In [1]:
import pandasfeatures = pandas.read_csv('./features.csv', index_col='match_id')
features = pandas.read_csv('./features.csv', index_col='match_id')

In [3]:
features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16,2449,0,4,1974,3,63


In [173]:
Xtr = features.drop(['start_time', 'radiant_win', 'tower_status_radiant', 'tower_status_dire', 'barracks_status_radiant', 'barracks_status_dire', 'duration'], 1)

In [174]:
l = len(Xtr)

In [175]:
diffs = np.subtract(l, Xtr.count())

In [176]:
sum(diffs)

193087

In [177]:
Xtr.columns[diffs != 0]

Index([u'first_blood_time', u'first_blood_team', u'first_blood_player1',
       u'first_blood_player2', u'radiant_bottle_time', u'radiant_courier_time',
       u'radiant_flying_courier_time', u'radiant_first_ward_time',
       u'dire_bottle_time', u'dire_courier_time', u'dire_flying_courier_time',
       u'dire_first_ward_time'],
      dtype='object')

In [178]:
Xtr.fillna(0, inplace=True)

In [179]:
ytr = features['radiant_win']

In [180]:
from sklearn.cross_validation import KFold
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.metrics import roc_auc_score
from sklearn.cross_validation import cross_val_score
import time
import datetime

kf = KFold(l, n_folds=5, shuffle=True)
times = []
q = []
for nCl in [5, 10, 15, 20, 25, 30]:
    clf = GBC(n_estimators=nCl)
    start_time = datetime.datetime.now()
    scores = cross_val_score(clf, Xtr, ytr, scoring='roc_auc', cv=kf)
    times.append(datetime.datetime.now() - start_time)
    q.append(scores.mean())

In [181]:
times

[datetime.timedelta(0, 11, 13822),
 datetime.timedelta(0, 19, 72330),
 datetime.timedelta(0, 26, 892979),
 datetime.timedelta(0, 34, 428118),
 datetime.timedelta(0, 43, 289389),
 datetime.timedelta(0, 50, 757020)]

In [182]:
q

[0.63658173831220122,
 0.66432838913622638,
 0.67537425886469515,
 0.68201226489900812,
 0.68625288700321352,
 0.68922597986198342]

In [183]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
Xsc = scaler.fit_transform(Xtr)

In [184]:
from sklearn.grid_search import GridSearchCV

grid = {'C': np.power(10.0, np.arange(-5, 6))}
kf = KFold(l, n_folds=5, shuffle=True)
clf = LogisticRegression()

gs = GridSearchCV(clf, grid, scoring='roc_auc', cv=kf)
gs.fit(Xsc, ytr)

GridSearchCV(cv=sklearn.cross_validation.KFold(n=97230, n_folds=5, shuffle=True, random_state=None),
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': array([  1.00000e-05,   1.00000e-04,   1.00000e-03,   1.00000e-02,
         1.00000e-01,   1.00000e+00,   1.00000e+01,   1.00000e+02,
         1.00000e+03,   1.00000e+04,   1.00000e+05])},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [185]:
gs.grid_scores_

[mean: 0.69508, std: 0.00489, params: {'C': 1.0000000000000001e-05},
 mean: 0.71118, std: 0.00404, params: {'C': 0.0001},
 mean: 0.71617, std: 0.00350, params: {'C': 0.001},
 mean: 0.71636, std: 0.00342, params: {'C': 0.01},
 mean: 0.71634, std: 0.00342, params: {'C': 0.10000000000000001},
 mean: 0.71634, std: 0.00342, params: {'C': 1.0},
 mean: 0.71633, std: 0.00342, params: {'C': 10.0},
 mean: 0.71633, std: 0.00342, params: {'C': 100.0},
 mean: 0.71633, std: 0.00342, params: {'C': 1000.0},
 mean: 0.71633, std: 0.00342, params: {'C': 10000.0},
 mean: 0.71633, std: 0.00342, params: {'C': 100000.0}]

In [186]:
noCatX = Xtr.drop(Xtr.filter(regex='[rd]._hero').columns,1).drop('lobby_type', 1)

In [187]:
noCatXsc = scaler.fit_transform(noCatX)

In [188]:
gs.fit(noCatXsc, ytr)

GridSearchCV(cv=sklearn.cross_validation.KFold(n=97230, n_folds=5, shuffle=True, random_state=None),
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': array([  1.00000e-05,   1.00000e-04,   1.00000e-03,   1.00000e-02,
         1.00000e-01,   1.00000e+00,   1.00000e+01,   1.00000e+02,
         1.00000e+03,   1.00000e+04,   1.00000e+05])},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [189]:
gs.grid_scores_

[mean: 0.69503, std: 0.00489, params: {'C': 1.0000000000000001e-05},
 mean: 0.71117, std: 0.00406, params: {'C': 0.0001},
 mean: 0.71622, std: 0.00354, params: {'C': 0.001},
 mean: 0.71642, std: 0.00346, params: {'C': 0.01},
 mean: 0.71640, std: 0.00346, params: {'C': 0.10000000000000001},
 mean: 0.71639, std: 0.00346, params: {'C': 1.0},
 mean: 0.71639, std: 0.00346, params: {'C': 10.0},
 mean: 0.71639, std: 0.00347, params: {'C': 100.0},
 mean: 0.71639, std: 0.00346, params: {'C': 1000.0},
 mean: 0.71639, std: 0.00346, params: {'C': 10000.0},
 mean: 0.71639, std: 0.00346, params: {'C': 100000.0}]

In [190]:
uni = np.unique(Xtr.filter(regex=r"^[rd]\d_hero$").values)

In [191]:
len(uni)

108

In [ ]:
heroes = {}
for i in range(len(uni)):
    heroes[uni[i]] = i

In [193]:
X_pick = np.zeros((features.shape[0], 108))

for i, match_id in enumerate(features.index):
    for p in xrange(5):
        X_pick[i, heroes[features.ix[match_id, 'r%d_hero' % (p+1)]]] = 1
        X_pick[i, heroes[features.ix[match_id, 'd%d_hero' % (p+1)]]] = -1

In [195]:
noCatXscHs = np.hstack([noCatXsc, X_pick])

In [196]:
gs.fit(noCatXscHs, ytr)

GridSearchCV(cv=sklearn.cross_validation.KFold(n=97230, n_folds=5, shuffle=True, random_state=None),
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': array([  1.00000e-05,   1.00000e-04,   1.00000e-03,   1.00000e-02,
         1.00000e-01,   1.00000e+00,   1.00000e+01,   1.00000e+02,
         1.00000e+03,   1.00000e+04,   1.00000e+05])},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [197]:
gs.grid_scores_

[mean: 0.69914, std: 0.00479, params: {'C': 1.0000000000000001e-05},
 mean: 0.72491, std: 0.00363, params: {'C': 0.0001},
 mean: 0.74619, std: 0.00215, params: {'C': 0.001},
 mean: 0.75167, std: 0.00141, params: {'C': 0.01},
 mean: 0.75188, std: 0.00129, params: {'C': 0.10000000000000001},
 mean: 0.75187, std: 0.00127, params: {'C': 1.0},
 mean: 0.75187, std: 0.00127, params: {'C': 10.0},
 mean: 0.75187, std: 0.00127, params: {'C': 100.0},
 mean: 0.75187, std: 0.00127, params: {'C': 1000.0},
 mean: 0.75187, std: 0.00127, params: {'C': 10000.0},
 mean: 0.75187, std: 0.00127, params: {'C': 100000.0}]

In [198]:
Xtest = pandas.read_csv('./features_test.csv', index_col='match_id')

In [199]:
noCatTest = Xtest.drop(Xtest.filter(regex='[rd]._hero').columns,1).drop('lobby_type', 1).drop('start_time',1)

In [200]:
noCatTest.fillna(0, inplace=True)

In [205]:
noCatTestSc = scaler.transform(noCatTest)

In [209]:
X_pick_test = np.zeros((Xtest.shape[0], 108))

for i, match_id in enumerate(Xtest.index):
    for p in xrange(5):
        X_pick_test[i, heroes[Xtest.ix[match_id, 'r%d_hero' % (p+1)]]] = 1
        X_pick_test[i, heroes[Xtest.ix[match_id, 'd%d_hero' % (p+1)]]] = -1

In [213]:
noCatTestScHs = np.hstack([noCatTestSc, X_pick_test])

In [215]:
optClf = LogisticRegression(C=0.1)
optClf.fit(noCatXscHs, ytr)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [216]:
preds = optClf.predict_proba(noCatTestScHs)[:, 1]

In [217]:
preds

array([ 0.83541878,  0.77379803,  0.2021713 , ...,  0.22879715,
        0.61545847,  0.41810845])

In [218]:
min(preds)

0.0083485828805909199

In [219]:
max(preds)

0.99653580085467541